In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
import numpy as np
import torch.utils.data as Data
from PIL import Image

import tools
import torch
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True)

In [4]:
import pandas as pd
df=pd.read_csv("../HAM/HAM10000_metadata.csv")
df=df[["image_id","dx"]]
mapping = {
    "nv": 0,
    "mel": 1,
    "bkl":2,
    "bcc": 3,
    "vasc": 4,
    "df": 5,
    "akiec": 6
}

# Map values directly
df["Hlabels"] = df["dx"].map(mapping)

In [5]:
from PIL import Image
import os
import numpy as np
import os
from PIL import Image
from torchvision import transforms
from tqdm import tqdm  # Progress bar for large datasets

# Folder containing the .tif images
folder_path = '../HAM/images/'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
])
# List to store images
Himages = []
Hlabels=[]
no0=0;
# Iterate through the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):
        if(int(df[df["image_id"]==str(filename[0:-4])]["Hlabels"]) in [0,1,2,3]):
            if(int(df[df["image_id"]==str(filename[0:-4])]["Hlabels"])==0):
                no0=no0+1
                if(no0>1200):
                    continue
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert('RGB')
    #         img = transform(img)
            img = np.array(img).flatten()
            Himages.append(np.array(img))
            filename=filename[0:-4]
            Hlabels.append(int(df[df["image_id"]==str(filename)]["Hlabels"]))
                
Himages = np.array(Himages) 
Hlabels=np.array(Hlabels)
# Check the number of images loaded
print(f"Loaded {len(Himages)} images.")

Loaded 3926 images.


In [6]:
Hlabels=torch.from_numpy(Hlabels).to(device)

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

# Split the dataset into training (80%) and testing (20%)
trainimg, test_images, trla, test_labels = train_test_split(Himages, Hlabels, test_size=0.12, random_state=42)
Himages, val_images, labels, vallabels = train_test_split(trainimg, trla, test_size=0.14, random_state=42)

In [8]:
data1=torch.from_numpy(Himages).float().to(device)
targets=labels

In [9]:
# original_images = np.load('data/cifar10/train_images.npy')
# original_labels = np.load('data/cifar10/train_labels.npy') 
# data1 = torch.from_numpy(original_images).float().to(device)
# targets = torch.from_numpy(original_labels).to(device)
dataset = zip(data1, targets)
data1=Himages
# data1=data1.reshape(-1, 3, 32, 32)
# data1=data1.transpose((0, 2, 3, 1))


In [10]:
noise_rate = 0.2
bsize=16
lr=0.0001
num_classes = 4
feature_size = 224*224*3
norm_std = 0.1
seed = 1
new_labels1 = tools.get_instance_noisy_label(noise_rate, dataset, targets, num_classes, feature_size, norm_std, seed)

building dataset...


In [11]:
# import numpy as np
# import pickle

# def save_cifar_batch(images, labels, batch_label, filename):
#     """
#     Save images and labels in the CIFAR-like format using pickle.

#     Args:
#     - images: NumPy array of shape (N, 3, 32, 32).
#     - labels: List of integer labels.
#     - batch_label: String label for the batch.
#     - filename: Path where the pickle file will be saved.
#     """
#     # Flatten the images from (N, 3, 32, 32) to (N, 3072)
#     flattened_images = images.reshape(images.shape[0], -1)

#     # Create a dictionary similar to the CIFAR format
#     batch_dict = {
#         'data': flattened_images,
#         'labels': labels,
#         'batch_label': batch_label
#     }

#     # Save the dictionary as a pickle file
#     with open(filename, 'wb') as f:
#         pickle.dump(batch_dict, f)

# def split_and_save_batches(images, labels, num_batches=5, prefix="data_batch_"):
#     """
#     Split images and labels into several batches and save each as a pickle file.

#     Args:
#     - images: NumPy array of shape (N, 3, 32, 32).
#     - labels: List of integer labels.
#     - num_batches: Number of batches to split the data into.
#     - prefix: Prefix for the batch file names.
#     """
#     N = images.shape[0]
#     batch_size = N // num_batches

#     for i in range(num_batches):
#         start_idx = i * batch_size
#         end_idx = (i + 1) * batch_size if i != num_batches - 1 else N  # Include remaining data in the last batch
#         batch_images = images[start_idx:end_idx]
#         batch_labels = labels[start_idx:end_idx]
#         batch_label = f"training batch {i+1} of {num_batches}"

#         # Save the batch to a pickle file
#         filename = "./HAM0.2/"+f"{prefix}{i+1}.pkl"
#         save_cifar_batch(batch_images, batch_labels, batch_label, filename)
#         print(f"Saved {filename} with {len(batch_labels)} images.")



# # Split and save into 5 batches
# split_and_save_batches(data1, new_labels1, num_batches=5, prefix="data_batch_")
# split_and_save_batches(test_images, test_labels, num_batches=1, prefix="test_batch_")
# # test_images = np.load('data/cifar10/test_images.npy')
# # test_labels = np.load('data/cifar10/test_labels.npy')
# # testl = torch.from_numpy(test_labels)


# # split_and_save_batches(test_images, testl, num_batches=1, prefix="test_batch_")


In [12]:
data1=data1.reshape(-1, 3,224, 224)

# data1=data1.reshape(-1, 3, 600, 400)
data1=data1.transpose((0, 2, 3, 1))

test_images=test_images.reshape(-1, 3,224, 224)
test_images=test_images.transpose((0, 2, 3, 1))

val_images=val_images.reshape(-1, 3,224, 224)
val_images=val_images.transpose((0, 2, 3, 1))

In [13]:
data1.shape

(2970, 224, 224, 3)

In [14]:
# !rm -r checkpoint
# !mkdir checkpoint
# !rm -rf ./cifar10/*.json

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torchvision import models

In [17]:
# model=torch.load('./2024_08_14-05_29_56_PM/best_cp/modelk.pth',map_location='cuda:0')

In [18]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.models as models

# # Load the pretrained ResNet-50 model
# model = models.resnet50(pretrained=True)
# model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.fc = nn.Linear(model.fc.in_features, 10)
# # Freeze the initial half of the layers
# total_layers = len(list(model.children()))  # Total number of layers
# freeze_layers = total_layers // 2  # Freeze the first half

# for i, child in enumerate(model.children()):
#     if i < freeze_layers:
#         for param in child.parameters():
#             param.requires_grad = False
#     else:
#         for param in child.parameters():
#             param.requires_grad = True
# model.to(device)
# # Verify which layers are frozen
# for name, param in model.named_parameters():
#     print(f'{name}: {"Frozen" if not param.requires_grad else "Trainable"}')


In [19]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.models as models
# import torch.nn.init as init

# # Load the pretrained ResNet-50 model
# model = models.resnet50(pretrained=True)
# model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

# model.fc = nn.Linear(model.fc.in_features, 10)
# # Define a function to apply Xavier initialization
# def initialize_weights_xavier(model):
#     for m in model.modules():
#         if isinstance(m, (nn.Conv2d, nn.Linear)):  # Apply Xavier to Conv2d and Linear layers
#             init.xavier_uniform_(m.weight)  # Xavier uniform distribution
#             if m.bias is not None:  # If bias exists, initialize it to 0
#                 init.constant_(m.bias, 0)

# # Apply Xavier initialization to the model
# initialize_weights_xavier(model)
# model.to(device)
# # Check the model to confirm initialization
# print(model)


In [20]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.models as models


# model = models.resnet50(pretrained=True)  


# # model.load_state_dict(torch.load('./2024_08_14-05_29_56_PM/best_cp/modelk.pth',map_location='cuda:0'),strict=False)
# model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

# model.fc = nn.Linear(model.fc.in_features, 10)
# model.to(device)
# # optimizer = optim.Adam(model.parameters(), lr=0.001)


# # optimizer.load_state_dict(torch.load('./2024_08_14-05_29_56_PM/best_cp/optimizer.pth',map_location='cuda:0'))

# # # Initialize Predictor (if it's also a ResNet-18 or another model)
# # predictor = models.resnet18(pretrained=False)
# # predictor.load_state_dict(torch.load('./2024_08_14-05_29_56_PM/last_cp/predictor.pth'))




In [21]:
# import numpy as np

# # Assuming images are in shape (N, 224, 224, 3) and dtype uint8
# # Normalize to [0,1] before computing mean/std
# images = data1.astype(np.float32) / 255.0

# # Compute mean and std along (0,1,2) axes (over all pixels)
# mean = images.mean(axis=(0, 1, 2))
# std = images.std(axis=(0, 1, 2))

# print(f"Mean: {mean.tolist()}")
# print(f"Std: {std.tolist()}")


In [22]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Adjusts crop scaling for large images
    transforms.ToTensor(),
    transforms.Normalize((0.6429, 0.5844, 0.6333), (0.1659, 0.1909, 0.1631)), 
])


In [23]:
from torch.utils.data import Dataset
from PIL import Image
class ExampleDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform  

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        x = Image.fromarray(x)
        y = self.labels[index]
        
        if self.transform:
            x = self.transform(x)  
            
        return x, y

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        x = Image.fromarray(x)
        if self.transform:
            x = self.transform(x)
        return x

In [24]:
def common_elements_same_index(arrays):
    if not arrays:
        return []

    # Number of arrays
    num_arrays = len(arrays)
    
    # Length of the first array
    num_elements = len(arrays[0])

    # Store indices where all arrays have the same element
    common_indices = []
    
    # Iterate over each index
    for i in range(num_elements):
        if all(array[i] == 1 for array in arrays):
            common_indices.append(i)
    
    return common_indices


# result = common_elements_same_index(confident_labels)



In [25]:
# len(result)

In [26]:
# itr1_pseudo=new_labels
def replace_elements_at_indices(another_array, common_indices, arrays):
    k=len(arrays)
    for i in common_indices:
        another_array[i] = arrays[k-1][i]
    
    return another_array

# itr1_pseudo = replace_elements_at_indices(itr1_pseudo, result, pseudo_labels)



In [27]:
# dataset = ExampleDataset(data, itr1_pseudo,transform=transform)
# dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True)

In [28]:
# # Step 3: Unfreeze all layers for further fine-tuning
# for param in model.parameters():
#     param.requires_grad = True  # Unfreeze all parameters

# # Redefine the optimizer to include all parameters now that all layers are unfrozen
# optimizer = optim.Adam(model.parameters(), lr=0.001)  # Use a smaller learning rate for fine-tuning


In [29]:
k_folds = 4
batch_size = 512

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset
from PIL import Image

# Data Augmentation (BYOL Transform for CIFAR-10)
class BYOLTransform:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(size=32),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize((0.6429, 0.5844, 0.6333), (0.1659, 0.1909, 0.1631)),  
        ])

    def __call__(self, x):
        return self.transform(x), self.transform(x)

# MLP for projection and prediction head
class MLP(nn.Module):
    def __init__(self, in_dim, hidden_dim=4096, out_dim=256):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, x):
        return self.net(x)

# BYOL model with backbone, projector, and predictor
class BYOL(nn.Module):
    def __init__(self, base_encoder, out_dim=256, momentum=0.996):
        super(BYOL, self).__init__()
        self.momentum = momentum
        
        # Online network
        self.online_encoder = base_encoder
        num_ftrs = self.online_encoder.fc.in_features
        self.online_encoder.fc = MLP(num_ftrs, out_dim=out_dim)
        
        # Target network (initialized with online network weights)
        self.target_encoder = base_encoder
        self.target_encoder.load_state_dict(self.online_encoder.state_dict())
        self.target_encoder.fc = MLP(num_ftrs, out_dim=out_dim)
        
        # Predictor
        self.predictor = MLP(out_dim, out_dim=out_dim)

    @torch.no_grad()
    def _momentum_update_target_encoder(self):
        # Momentum update for the target encoder
        for online_params, target_params in zip(self.online_encoder.parameters(), self.target_encoder.parameters()):
            target_params.data = self.momentum * target_params.data + (1.0 - self.momentum) * online_params.data

    def forward(self, x1, x2):
        # Online network forward pass
        online_proj_1 = self.online_encoder(x1)
        online_proj_2 = self.online_encoder(x2)
        
        # Predict from online projection
        pred_1 = self.predictor(online_proj_1)
        pred_2 = self.predictor(online_proj_2)
        
        # Target network forward pass (no gradient computation)
        with torch.no_grad():
            self._momentum_update_target_encoder()  # update the target encoder
            target_proj_1 = self.target_encoder(x1)
            target_proj_2 = self.target_encoder(x2)
        
        return pred_1, pred_2, target_proj_1, target_proj_2

# Loss function (mean squared error between predictions and targets)
def byol_loss_fn(pred, target):
    pred = F.normalize(pred, dim=-1, p=2)
    target = F.normalize(target, dim=-1, p=2)
    return 2 - 2 * (pred * target).sum(dim=-1).mean()

# CIFAR-10 Dataset class
class CIFAR10BYOLDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        x = Image.fromarray(x)
        y = self.labels[index]

        if self.transform:
            x_i, x_j = self.transform(x)
        else:
            x_i, x_j = x, x

        return (x_i, x_j), y

# Training function for BYOL with CIFAR-10
def train_byol(data_loader, model, optimizer, device='cuda'):
    model.train()
    for (x_i, x_j), _ in data_loader:
        x_i = x_i.to(device)
        x_j = x_j.to(device)
        
        # Forward pass
        pred_1, pred_2, target_1, target_2 = model(x_i, x_j)
        
        # Compute loss
        loss_1 = byol_loss_fn(pred_1, target_2)
        loss_2 = byol_loss_fn(pred_2, target_1)
        loss = (loss_1 + loss_2) / 2
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return loss.item()

class FineTunedModel(nn.Module):
    def __init__(self, pretrained_backbone, out_dim=10):
        super(FineTunedModel, self).__init__()
        self.backbone = pretrained_backbone  # Pretrained SSL backbone (ResNet)
        
        # Freeze the backbone parameters (optional, can be unfrozen later for fine-tuning)
        for param in self.backbone.parameters():
            param.requires_grad = False

        # New classification head
        self.classifier = nn.Sequential(
            nn.ReLU(),               # ReLU activation
            nn.Dropout(0.3),          # Dropout with 30% probability
            nn.Linear(256, out_dim)   # Classification layer with output size 10 (for CIFAR-10)
        )

    def forward(self, x):
        x = self.backbone(x)       # Pass input through the pretrained backbone
        x = self.classifier(x)     # Pass through the classifier head
        return x



In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader

# Data Augmentation (SimCLR Transform)
# class SimCLRTransform:
#     def __init__(self, size=224):
#         self.transform = transforms.Compose([
#             transforms.RandomResizedCrop(size=size),
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomApply([transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),
#             transforms.RandomGrayscale(p=0.2),
#             transforms.GaussianBlur(kernel_size=9),
#             transforms.ToTensor(),
#             transforms.Normalize((0.467, 0.459, 0.519), (0.207, 0.233, 0.261)),
#         ])

#     def __call__(self, x):
#         # Apply the transformation twice to create two views
#         return self.transform(x), self.transform(x)


SimCLRTransform=transforms.Compose([
            transforms.RandomResizedCrop(size=112),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.GaussianBlur(kernel_size=9),
            transforms.ToTensor(),
            transforms.Normalize((0.6429, 0.5844, 0.6333), (0.1659, 0.1909, 0.1631)),  
        ])

SimCLRTransform1=transforms.Compose([
            transforms.RandomResizedCrop(size=224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.GaussianBlur(kernel_size=9),
            transforms.ToTensor(),
            transforms.Normalize((0.6429, 0.5844, 0.6333), (0.1659, 0.1909, 0.1631)),   
        ])

# ResNet Backbone with Projection Head
class ResNetSimCLR(nn.Module):
    def __init__(self, out_dim=128):
        super(ResNetSimCLR, self).__init__()
        # Load a pre-trained ResNet-50 model
        self.resnet = models.resnet50(pretrained=True)
        
        # Remove the classification layer
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        
        # Projection head: 2-layer MLP
        self.projector = nn.Sequential(
            nn.Linear(num_ftrs, 512), 
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.projector(x)
        return x

# NT-Xent Loss (Contrastive Loss)
class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(NTXentLoss, self).__init__()
        self.temperature = temperature

    def forward(self, z_i, z_j):
        # Normalize the projections
        z_i = F.normalize(z_i, dim=1)
        z_j = F.normalize(z_j, dim=1)

        # Concatenate both projections
        batch_size = z_i.shape[0]
        representations = torch.cat([z_i, z_j], dim=0)

        # Compute cosine similarity matrix
        similarity_matrix = torch.matmul(representations, representations.T) / self.temperature

        # Create labels for positive pairs
        labels = torch.cat([torch.arange(batch_size) for _ in range(2)], dim=0).to(z_i.device)
        labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()

        # Mask out the diagonal (self-similarity)
        mask = torch.eye(labels.shape[0], dtype=torch.bool).to(z_i.device)
        labels = labels[~mask].view(labels.shape[0], -1)
        similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)

        # Compute loss
        positives = similarity_matrix[labels.bool()].view(batch_size, -1)
        negatives = similarity_matrix[~labels.bool()].view(batch_size, -1)

        logits = torch.cat([positives, negatives], dim=1)
        targets = torch.zeros(logits.shape[0], dtype=torch.long).to(z_i.device)

        loss = F.cross_entropy(logits, targets)
        return loss
    
class ExampleDataset1(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Get the original image
        x = self.data[index]
        x = Image.fromarray(x)
        y = self.labels[index]

        # Apply the SimCLR transformations (two different augmented views of the same image)
        if self.transform:
            x_i = self.transform(x)  # First view
            x_j = self.transform(x)  # Second view
        else:
            x_i, x_j = x, x  # In case no transformation is applied

        return (x_i, x_j), y  # Return both views and the label (y) if applicable


In [32]:
# test_images = np.load('data/cifar10/test_images.npy')
# test_labels = np.load('data/cifar10/test_labels.npy')
# from sklearn.model_selection import train_test_split
# # test_dataset = ExampleDataset(test_images, test_labels,transform)
# test_images=test_images.reshape(-1, 3, 32, 32)
# datatest=test_images
# datatest=datatest.reshape(-1, 3, 32, 32)
# datatest=datatest.transpose((0, 2, 3, 1))
# testl = torch.from_numpy(test_labels)

# val_images, test_images, vallabels, test_labels = train_test_split(
#     datatest, testl, test_size=0.4, random_state=42
# )


In [33]:
# augmentation_transforms = transforms.Compose([
#     transforms.RandomResizedCrop(size=32, scale=(0.8, 1.0)),  # Resize and crop
#     transforms.RandomHorizontalFlip(),  # Horizontal flip
#     transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),  # Color jitter
#     transforms.RandomGrayscale(p=0.2),  # Convert to grayscale randomly
#     transforms.GaussianBlur(kernel_size=3),  # Apply Gaussian blur
#     transforms.ToTensor()
# ])


import torchvision.transforms as transforms

augmentation_transforms = transforms.Compose([
        transforms.RandomRotation(degrees=15),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Adjusted for 224x224
        transforms.RandomHorizontalFlip(),                    # Horizontal flip
#         transforms.ColorJitter(brightness=0.2, contrast=0.2), # Color jitter
#         transforms.RandomGrayscale(p=0.1)                     # Grayscale randomly
        
        transforms.ToTensor()
])

In [34]:
# # for fold, (train_idx, val_idx) in enumerate(kf.split(data1)):
# from PreResNet import *

# def create_model():
#     model = ResNet18(num_classes=4)
#     model = model.cuda()
#     return model
# #     data = data1[train_idx]
# #     new_labels = new_labels1[train_idx]
# #     val_data = data1[val_idx]
# #     val_labels = targets[val_idx]
# fold = 0

# data = data1
# new_labels = new_labels1
# val_data = val_images
# val_labels = vallabels

# #     model = models.resnet50(pretrained=True)  
# # # model.load_state_dict(torch.load('./2024_08_14-05_29_56_PM/best_cp/modelk.pth',map_location='cuda:0'),strict=False)


# #     model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

# # model.to(device)
# # model.fc = nn.Linear(model.fc.in_features, 10)
# # model.to(device)


# # criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
# # criterion1 = nn.CrossEntropyLoss(reduction='none',label_smoothing=0.1)
# # optimizer = optim.Adam(model.parameters(), lr=0.01)
# # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)




# # # Model, Loss, and Optimizer
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# # resnet = models.resnet18(pretrained=False)
# resnet = create_model()
# # resnet.load_state_dict(torch.load('net1withoutssl0.5.pth'))
# # num_ftrs = resnet.linear.in_features
# # resnet.linear = nn.Identity()  # Replace the FC layer with an Identity layer
# # model = nn.Sequential(
# #     resnet,                    # ResNet-50 backbone without classification layer
# #     nn.Linear(num_ftrs, 512),   # First linear layer of the projection head
# #     nn.ReLU(),                  # ReLU activation function
# #     nn.Linear(512, 128)         # Second linear layer projecting to 128 dimensions
# # )
# model=resnet
# model.to(device)







# # num_ftrs = resnet.fc.in_features
# # resnet.fc = nn.Identity()  # Replace the FC layer with an Identity layer
# # model = nn.Sequential(
# #     resnet,                    # ResNet-50 backbone without classification layer
# #     nn.Linear(num_ftrs, 512),   # First linear layer of the projection head
# #     nn.ReLU(),                  # ReLU activation function
# #     nn.Linear(512, 128)         # Second linear layer projecting to 128 dimensions
# # )
# # model.to(device)



# # model = models.resnet50(pretrained=False)
# # model.load_state_dict(torch.load('resnet50_weights.pth'))







# criterion = NTXentLoss(temperature=0.3)
# # optimizer = optim.Adam(model.parameters(), lr=3e-4)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.95)

# dataset = ExampleDataset1(data, new_labels,transform=SimCLRTransform)
# dataloader = DataLoader(dataset, batch_size=64, shuffle=True,num_workers=4)
# epochs = 15
# for epoch in range(epochs):
#     model.train()
#     running_loss = 0.0

#     for images, _ in dataloader:
#         img_1, img_2 = images  # Two augmented views of the same image

#         # Move to device
#         img_1, img_2 = img_1.to(device), img_2.to(device)

#         # Forward pass
#         z_i = model(img_1)
#         z_j = model(img_2)

#         # Compute loss
#         loss = criterion(z_i, z_j)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(dataloader)}')

    
    
# dataset = ExampleDataset1(data, new_labels,transform=SimCLRTransform1)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True,num_workers=4)
# epochs = 15
# for epoch in range(epochs):
#     model.train()
#     running_loss = 0.0

#     for images, _ in dataloader:
#         img_1, img_2 = images  # Two augmented views of the same image

#         # Move to device
#         img_1, img_2 = img_1.to(device), img_2.to(device)

#         # Forward pass
#         z_i = model(img_1)
#         z_j = model(img_2)

#         # Compute loss
#         loss = criterion(z_i, z_j)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(dataloader)}')

    
    
    
# torch.save(model.state_dict(), 'HAMsslonly.pth')

In [35]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class RobustLoss(nn.Module):
#     def __init__(self, alpha=0.25, gamma=2.0, label_smoothing=0.1, reduction='mean'):
#         super(RobustLoss, self).__init__()
#         self.alpha = alpha  # Weighting factor for balancing positive/negative class
#         self.gamma = gamma  # Focusing parameter
#         self.label_smoothing = label_smoothing
#         self.reduction = reduction

#     def forward(self, outputs, targets):
#         # Apply label smoothing
#         num_classes = outputs.size(1)
#         one_hot_targets = torch.zeros_like(outputs).scatter_(1, targets.unsqueeze(1), 1)
#         smooth_target = (1 - self.label_smoothing) * one_hot_targets + self.label_smoothing / num_classes

#         # Compute cross-entropy loss
#         log_p = F.log_softmax(outputs, dim=1)
#         loss = -smooth_target * log_p

#         # Focal Loss
#         p_t = torch.exp(-loss)  # Probability of the true class
#         focal_loss = self.alpha * (1 - p_t) ** self.gamma * loss

#         # Reduce the loss
#         if self.reduction == 'mean':
#             return focal_loss.mean()
#         elif self.reduction == 'sum':
#             return focal_loss.sum()
#         else:
#             return focal_loss



In [36]:
# import torch
# from torch.utils.data import Dataset
# from PIL import Image
# import numpy as np
# from torch.utils.data import DataLoader, TensorDataset

# class NDataset(Dataset):
#     def __init__(self, images, labels, transform=None, num_classes=10):
#         """
#         A dataset class that maintains label confidence scores for noisy datasets.

#         Args:
#             images (list or numpy.ndarray): The dataset of images.
#             labels (list or numpy.ndarray): The noisy labels corresponding to the images.
#             transform (callable, optional): A function/transform to apply to the images.
#             num_classes (int, optional): Number of classes in the dataset (default is 10).
#         """
#         self.images = images
#         self.labels = np.array(labels)
#         self.transform = transform
#         self.num_classes = num_classes

#         # Initialize label confidence scores uniformly
#         self.label_confidence = torch.ones(len(self.labels), self.num_classes) / self.num_classes

#     def update_confidence(self, model, device, m=0, batch_size=bsize, num_workers=4):
#         """
#         Updates label confidence scores based on the given mathematical expression.

#         Args:
#             model (torch.nn.Module): The trained model.
#             device (torch.device): The device to run the model on (e.g., 'cpu' or 'cuda').
#             criterion: Loss function to compute L(f(x; θ), y).
#             m (float): A variable that affects the size of |Dce|.
#             batch_size (int): The number of images per batch.
#             num_workers (int): Number of subprocesses for data loading (default: 4).
#         """
#         model = model.to(device)
#         model.eval()

#         dataset = ExampleDataset(self.images, self.labels,self.transform)
#         data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
#         criterion = nn.CrossEntropyLoss(label_smoothing=0.1, reduction='none')
        

#         total_loss = 0.0
#         num_samples = 0
#         losses = []

#         # Compute losses for the entire dataset
#         with torch.no_grad():
#             for batch_images, batch_labels in data_loader:
#                 batch_images = batch_images.to(device)
#                 batch_labels = batch_labels.to(device)


#                 # Forward pass
#                 outputs = model(batch_images)
# #                     batch_labels = torch.tensor(self.labels[num_samples:num_samples + batch_images.size(0)], device=device)

#                 # Compute loss
#                 loss = criterion(outputs, batch_labels)
#                 losses.append(loss)
#                 total_loss += loss.sum().item()
#                 num_samples += batch_images.size(0)

#         # Compute mean loss µ
#         mu = total_loss / num_samples

#         # Update confidence scores using the formula
#         with torch.no_grad():
#             for batch_idx, (batch_images,batch_labels) in enumerate(data_loader):
#                 batch_images = batch_images.to(device)
#                 batch_labels = batch_labels.to(device)

#                 # Forward pass
#                 outputs = model(batch_images)
# #                     batch_labels = torch.tensor(self.labels[batch_idx * batch_size : batch_idx * batch_size + batch_images.size(0)], device=device)

#                 # Compute loss
#                 loss = criterion(outputs, batch_labels)

#                 # Compute h(f(x;θ)y) using the given formula
#                 h_values = torch.sigmoid(0.5 * (-loss + mu + m))

#                 # Update confidence scores
#                 start_idx = batch_idx * batch_size
#                 end_idx = start_idx + batch_images.size(0)
#                 self.label_confidence[start_idx:end_idx] = h_values.cpu().unsqueeze(1).repeat(1, self.num_classes)


#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         """
#         Returns the image, label, and confidence scores at the given index.

#         Args:
#             idx (int): The index of the data point.

#         Returns:
#             tuple: (image, label, confidence_scores)
#         """
#         image = Image.fromarray(self.images[idx])
#         if self.transform:
#             image = self.transform(image)
#         label = self.labels[idx]
#         confidence = self.label_confidence[idx]
#         return image, label, confidence



# # Step 4: Define Training Loss Function with Confidence Incorporation
# class ConfidenceWeightedLoss(nn.Module):
#     def __init__(self):
#         super(ConfidenceWeightedLoss, self).__init__()
#         self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1, reduction='none')
        

#     def forward(self, outputs, targets, confidences):
#         losses = self.criterion(outputs, targets)
#         log_probs = F.log_softmax(outputs, dim=1)
#         Deno = torch.sum(log_probs, dim=1)
#         losses1=-losses/(Deno+1e-8)
#         weighted_loss = (losses1+losses * confidences[range(len(targets)), targets]).mean()
#         return weighted_loss

In [37]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

class NDataset(Dataset):
    def __init__(self, images, labels, transform=None, num_classes=4):
        """
        A dataset class that maintains label confidence scores for noisy datasets.

        Args:
            images (list or numpy.ndarray): The dataset of images.
            labels (list or numpy.ndarray): The noisy labels corresponding to the images.
            transform (callable, optional): A function/transform to apply to the images.
            num_classes (int, optional): Number of classes in the dataset (default is 10).
        """
        self.images = images
        self.labels = np.array(labels)
        self.transform = transform
        self.num_classes = num_classes

        # Initialize label confidence scores uniformly
        self.label_confidence = torch.ones(len(self.labels), self.num_classes) / self.num_classes

    def update_confidence(self, model, device, m=0, batch_size=bsize, num_workers=4):
        """
        Updates label confidence scores based on the given mathematical expression.

        Args:
            model (torch.nn.Module): The trained model.
            device (torch.device): The device to run the model on (e.g., 'cpu' or 'cuda').
            criterion: Loss function to compute L(f(x; θ), y).
            m (float): A variable that affects the size of |Dce|.
            batch_size (int): The number of images per batch.
            num_workers (int): Number of subprocesses for data loading (default: 4).
        """
        model = model.to(device)
        model.eval()

        dataset = ExampleDataset(self.images, self.labels,self.transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
        criterion = nn.CrossEntropyLoss(label_smoothing=0.1, reduction='none')
        

#         total_loss = 0.0
#         num_samples = 0
#         losses = []

#         # Compute losses for the entire dataset
#         with torch.no_grad():
#             for batch_images, batch_labels in data_loader:
#                 batch_images = batch_images.to(device)
#                 batch_labels = batch_labels.to(device)


#                 # Forward pass
#                 outputs = model(batch_images)
# #                     batch_labels = torch.tensor(self.labels[num_samples:num_samples + batch_images.size(0)], device=device)

#                 # Compute loss
#                 loss = criterion(outputs, batch_labels)
#                 losses.append(loss)
#                 total_loss += loss.sum().item()
#                 num_samples += batch_images.size(0)

#         # Compute mean loss µ
#         mu = total_loss / num_samples

        # Update confidence scores using the formula
        with torch.no_grad():
            for batch_idx, (batch_images,batch_labels) in enumerate(data_loader):
                batch_images = batch_images.to(device)
                batch_labels = batch_labels.to(device)

                # Forward pass
                outputs = model(batch_images)
#                     batch_labels = torch.tensor(self.labels[batch_idx * batch_size : batch_idx * batch_size + batch_images.size(0)], device=device)
                probs = torch.softmax(outputs, dim=1)
                # Compute loss
#                 loss = criterion(outputs, batch_labels)

#                 # Compute h(f(x;θ)y) using the given formula
#                 h_values = torch.sigmoid(0.5 * (-loss + mu + m))

                # Update confidence scores
                start_idx = batch_idx * batch_size
                end_idx = start_idx + batch_images.size(0)
                self.label_confidence[start_idx:end_idx] = probs.cpu()


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        """
        Returns the image, label, and confidence scores at the given index.

        Args:
            idx (int): The index of the data point.

        Returns:
            tuple: (image, label, confidence_scores)
        """
        image = Image.fromarray(self.images[idx])
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        confidence = self.label_confidence[idx]
        return image, label, confidence



# Step 4: Define Training Loss Function with Confidence Incorporation
class ConfidenceWeightedLoss(nn.Module):
    def __init__(self):
        super(ConfidenceWeightedLoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()
        

    def forward(self, outputs, targets, confidences):
        losses = self.criterion(outputs, targets)
#         log_probs = F.log_softmax(outputs, dim=1)
#         Deno = torch.sum(log_probs, dim=1)
#         losses1=-losses/(Deno+1e-8)
#         weighted_loss = (losses * confidences[range(len(targets)), targets]).mean()
        return losses

In [38]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score

def dynamic_loss_thresholding(losses, labels, percentile=30):
    """Computes class-wise dynamic loss threshold based on the given percentile while preserving order."""
    thresholds = {}
    reliable_indices = torch.zeros_like(losses, dtype=torch.bool)
    
    unique_classes = torch.unique(labels)
    for cls in unique_classes:
        class_mask = labels == cls
        class_losses = losses[class_mask]
        if len(class_losses) > 0:
            threshold = np.percentile(class_losses.cpu().numpy(), percentile)
            thresholds[cls.item()] = threshold
    
    for i in range(len(losses)):
        cls = labels[i].item()
        if cls in thresholds and losses[i] < thresholds[cls]:
            reliable_indices[i] = True
    
    return reliable_indices

def pseudo_labeling(model, dataloader, criterion1, device, percentile=30):
    """Generates pseudo-labels and selects reliable samples using dynamic loss thresholding while preserving order."""
    model.eval()
    all_predictions = []
    all_losses = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            loss_per_sample = criterion1(outputs, labels)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_losses.extend(loss_per_sample.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    all_predictions = np.array(all_predictions)
    all_losses = torch.tensor(all_losses, device=device)
    all_labels = torch.tensor(all_labels, device=device)
    
    reliable_indices = dynamic_loss_thresholding(all_losses, all_labels, percentile)
    reliable_indices_np = reliable_indices.cpu().numpy().astype(int)  # Convert to binary 1/0
    
    return all_predictions, reliable_indices_np

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # Weighting factor for class imbalance
        self.gamma = gamma  # Focusing parameter
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction="none")  # Compute cross-entropy loss
        pt = torch.exp(-ce_loss)  # Probabilities of correct class
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss  # Apply focal weight
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss  # No reduction

# Example usage



In [40]:
# for fold, (train_idx, val_idx) in enumerate(kf.split(data1)):
from PreResNet import *
dmp=2

def create_model():
    model = ResNet18(num_classes=4)
    model = model.cuda()
    return model

fold = 0

data = data1
new_labels = new_labels1
val_data = val_images
val_labels = vallabels



# # Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# resnet = models.resnet18(pretrained=False)
resnet = create_model()
# resnet.load_state_dict(torch.load('net1SSLcifar100.3.pth'))
# resnet.load_state_dict(torch.load('net1SSLcifar100.3.pth'))
# resnet.load_state_dict(torch.load('net1sslafterdividemix.pth'))
# resnet.load_state_dict(torch.load('sslthendividemix.pth'))
resnet.load_state_dict(torch.load('net1HAM0.2.pth'))
# resnet.load_state_dict(torch.load('chestxsslonlyfewbatches.pth'))

num_ftrs = resnet.linear.in_features
resnet.linear = nn.Identity()  # Replace the FC layer with an Identity layer
model = nn.Sequential(
    resnet,                    # ResNet-50 backbone without classification layer
    nn.Linear(num_ftrs, 512),   # First linear layer of the projection head
    nn.ReLU(),                  # ReLU activation function
    nn.Linear(512, 128)         # Second linear layer projecting to 128 dimensions
)





#simclr
model = nn.Sequential(
    model,                    # ResNet-50 backbone without classification layer
    nn.ReLU(),                  # ReLU activation function
#     nn.Dropout(0.7),
    nn.Linear(128, 4)         # Second linear layer projecting to 128 dimensions
)


model.to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
criterion1 = nn.CrossEntropyLoss(reduction='none')
training_criterion = ConfidenceWeightedLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.95)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.95,weight_decay=5e-4)


criterion = FocalLoss(alpha=0.25, gamma=2.0)
criterion1 = FocalLoss(alpha=0.25, gamma=2.0,reduction='none')


dataset = NDataset(data, new_labels,transform=transform)
dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True,num_workers=4)

num_epochs = 5

for epoch in range(num_epochs):
#     dataset.update_confidence(model, device)
    model.train()
    running_loss = 0.0
    for inputs, targets, confidences in dataloader:
        inputs = inputs.to(device)
        targets=targets.to(device)
        confidences=confidences.to(device)
        optimizer.zero_grad() 

        outputs = model(inputs)  
        loss = criterion(outputs, targets)
# #         confidences = torch.softmax(outputs, dim=1)
#         loss = training_criterion(outputs, targets, confidences)

        loss.backward() 
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()  
        # scheduler.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}')


val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
# model.train()
from sklearn.metrics import f1_score
f1 = f1_score(all_labels, all_predictions, average='macro')
print(f'Fold {fold + 1} F1 Score: {f1:.4f}')




# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.95)


stages = [2,3,4]
# stages=[2,4,7,9,11]
datasetpred = ExampleDataset(data, new_labels,transform=transform)
dataloaderpred = DataLoader(datasetpred, batch_size=bsize, shuffle=False,num_workers=4)
pseudo_labels=[]
confident_labels=[]
last_epoch=stages[-1]
ep=0;
model.train()
for st in stages:
    num_epochs = st;
    for epoch in range(ep,num_epochs):
#         dataset.update_confidence(model, device)
        running_loss = 0.0
        for inputs, targets, confidences in dataloader:
            inputs = inputs.to(device)
            targets=targets.to(device) 
            confidences=confidences.to(device)
            optimizer.zero_grad()  # Zero the gradients

            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets)  # Compute loss
# #             confidences = torch.softmax(outputs, dim=1)
#             loss = training_criterion(outputs, targets, confidences)

            loss.backward()  # Backward pass
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()  # Optimize
            # scheduler.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{last_epoch}], Loss: {running_loss / len(dataloader):.4f}')
    all_predictions = []
    reliable_indice=[]
    model.eval()
#     pred, reliable_indices = pseudo_labeling(model, dataloaderpred, criterion1, device)
#     pseudo_labels.append(pred)
#     confident_labels.append(reliable_indices)

    with torch.no_grad():
        for inputs, labels in dataloaderpred: 
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass to get outputs
            outputs = model(inputs)

            # Get the predictions
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())

            # Calculate per-sample loss
            loss_per_sample = criterion1(outputs, labels) 
            
            
            
            
            # print(loss_per_sample)
            reliable_indices = loss_per_sample < 0.25  # Example threshold for loss
            # print(reliable_indices)
            # Extend reliable indices list with boolean values indicating reliability
            reliable_indice.extend(reliable_indices.cpu().numpy())


    pseudo_labels.append(np.array(all_predictions))
    confident_labels.append(np.array(reliable_indice))
    ep=st;




val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
# model.train()
from sklearn.metrics import f1_score
f1 = f1_score(all_labels, all_predictions, average='macro')
print(f'Fold {fold + 1} F1 Score: {f1:.4f}')



result = common_elements_same_index(confident_labels)
print(len(result))
itr1_pseudo=new_labels
itr1_pseudo = replace_elements_at_indices(itr1_pseudo, result, pseudo_labels)

augmented_data = []
augmented_labels = []
for idx in result:
    input_data = data[idx]  # Get the data for the current index
    input_data = Image.fromarray(input_data)
    for _ in range(dmp):  # Perform 5 augmentations
        augmented_input = augmentation_transforms(input_data)  # Apply augmentations
        augmented_input = (augmented_input.permute(1, 2, 0).numpy()*255).astype(np.uint8)
        augmented_data.append(augmented_input)
        augmented_labels.append(itr1_pseudo[idx])  # Use the pseudo-label for augmented data

augmented_labels = np.array(augmented_labels)

# Concatenate the original data and augmented data
data = np.concatenate((data, np.array(augmented_data)), axis=0)
itr1_pseudo = np.concatenate((itr1_pseudo, augmented_labels), axis=0)


dataset = NDataset(data, itr1_pseudo,transform=transform)
dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True,num_workers=4)
datasetpred = ExampleDataset(data, itr1_pseudo,transform=transform)
dataloaderpred = DataLoader(datasetpred, batch_size=bsize, shuffle=False,num_workers=4)
p=[]
a=[]
optimizer = optim.Adam(model.parameters(), lr=0.00001)
augmented_epochs=[1,3,5]
for i in range(3):
    print(f'Iteration: {i}')
    stages = [2,5,7]
#     stages = [3,5,6,8,10]
    pseudo_labels1=[]
    confident_labels=[]
    last_epoch=stages[-1]
    ep=0;
    for st in stages:
        model.train()
        num_epochs = st;
        for epoch in range(ep,num_epochs):
#             dataset.update_confidence(model, device)
            running_loss = 0.0
            for inputs, targets, confidences in dataloader:
                inputs = inputs.to(device)
                targets=targets.to(device) 
                confidences=confidences.to(device)
                optimizer.zero_grad()  # Zero the gradients

                outputs = model(inputs)  # Forward pass
                loss = criterion(outputs, targets)  # Compute loss
# #                 confidences = torch.softmax(outputs, dim=1)
#                 loss = training_criterion(outputs, targets, confidences)

                loss.backward()  # Backward pass
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()  # Optimize
                # scheduler.step()
                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{last_epoch}], Loss: {running_loss / len(dataloader):.4f}')
        all_predictions = []
        reliable_indice=[]
        model.eval()
#         pred, reliable_indices = pseudo_labeling(model, dataloaderpred, criterion1, device)
#         pseudo_labels1.append(pred)
#         confident_labels.append(reliable_indices)
        with torch.no_grad():
            for inputs, labels in dataloaderpred: 
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass to get outputs
                outputs = model(inputs)

                # Get the predictions
                _, predicted = torch.max(outputs, 1)
                all_predictions.extend(predicted.cpu().numpy())

                # Calculate per-sample loss
                loss_per_sample = criterion1(outputs, labels) 

            
            
                # Set a loss threshold (lower loss means more reliable)
                reliable_indices = loss_per_sample < 0.25  # Example threshold for loss

                # Extend reliable indices list with boolean values indicating reliability
                reliable_indice.extend(reliable_indices.cpu().numpy())

        pseudo_labels1.append(np.array(all_predictions))
        confident_labels.append(np.array(reliable_indice))
        ep=st;
    result1 = common_elements_same_index(confident_labels)
    p.append(result1)
    a.append(pseudo_labels1)
    print(len(result1))
    itr1_pseudo = replace_elements_at_indices(itr1_pseudo, result1, pseudo_labels1)
    
    if i<2:
        augmented_data = []
        augmented_labels = []
        for idx in result1:
            input_data = data[idx]  # Get the data for the current index
            input_data = Image.fromarray(input_data)
            for _ in range(dmp):  # Perform 5 augmentations
                augmented_input = augmentation_transforms(input_data)  # Apply augmentations
                augmented_input = (augmented_input.permute(1, 2, 0).numpy()*255).astype(np.uint8)
                augmented_data.append(augmented_input)
                augmented_labels.append(itr1_pseudo[idx])  # Use the pseudo-label for augmented data

        augmented_labels = np.array(augmented_labels)

        # Concatenate the original data and augmented data
        data = np.concatenate((data, np.array(augmented_data)), axis=0)
        itr1_pseudo = np.concatenate((itr1_pseudo, augmented_labels), axis=0)
    
    
    
    dataset = NDataset(data, itr1_pseudo,transform=transform)
    dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True,num_workers=4)
    datasetpred = ExampleDataset(data, itr1_pseudo,transform=transform)
    dataloaderpred = DataLoader(datasetpred, batch_size=bsize, shuffle=False,num_workers=4)



    val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    # model.train()
    from sklearn.metrics import f1_score
    f1 = f1_score(all_labels, all_predictions, average='macro')
    print(f'Fold {fold + 1} F1 Score: {f1:.4f}')
    if(i==1):
        optimizer = optim.Adam(model.parameters(), 0.000001)

net1=model


Epoch [1/5], Loss: 0.1772
Epoch [2/5], Loss: 0.1707
Epoch [3/5], Loss: 0.1690
Epoch [4/5], Loss: 0.1644
Epoch [5/5], Loss: 0.1664
Fold 1 F1 Score: 0.4294
Epoch [1/4], Loss: 0.1659
Epoch [2/4], Loss: 0.1643
Epoch [3/4], Loss: 0.1815
Epoch [4/4], Loss: 0.1718
Fold 1 F1 Score: 0.2862
1992
Iteration: 0
Epoch [1/7], Loss: 0.1167
Epoch [2/7], Loss: 0.0984
Epoch [3/7], Loss: 0.0955
Epoch [4/7], Loss: 0.0943
Epoch [5/7], Loss: 0.0932
Epoch [6/7], Loss: 0.0932
Epoch [7/7], Loss: 0.0929
6079
Fold 1 F1 Score: 0.2457
Iteration: 1
Epoch [1/7], Loss: 0.0524
Epoch [2/7], Loss: 0.0508
Epoch [3/7], Loss: 0.0484
Epoch [4/7], Loss: 0.0478
Epoch [5/7], Loss: 0.0473
Epoch [6/7], Loss: 0.0461
Epoch [7/7], Loss: 0.0457
18111
Fold 1 F1 Score: 0.2372
Iteration: 2
Epoch [1/7], Loss: 0.0290
Epoch [2/7], Loss: 0.0278
Epoch [3/7], Loss: 0.0277
Epoch [4/7], Loss: 0.0275
Epoch [5/7], Loss: 0.0273
Epoch [6/7], Loss: 0.0273
Epoch [7/7], Loss: 0.0272
54216
Fold 1 F1 Score: 0.2400


In [41]:
# # data = data1
# # new_labels = new_labels1
# val_data = test_images
# val_labels = test_labels
# val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
# val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
# model.eval()
# all_predictions = []
# all_labels = []

# with torch.no_grad():
#     for inputs, labels in val_loader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         all_predictions.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())
# # model.train()
# from sklearn.metrics import f1_score
# f1 = f1_score(all_labels, all_predictions, average='macro')
# print(f'Fold {fold + 1} F1 Score: {f1:.4f}')

In [42]:
# for fold, (train_idx, val_idx) in enumerate(kf.split(data1)):
from PreResNet import *

def create_model():
    model = ResNet18(num_classes=4)
    model = model.cuda()
    return model
#     data = data1[train_idx]
#     new_labels = new_labels1[train_idx]
#     val_data = data1[val_idx]
#     val_labels = targets[val_idx]
fold = 0

data = data1
new_labels = new_labels1
val_data = val_images
val_labels = vallabels

#     model = models.resnet50(pretrained=True)  
# # model.load_state_dict(torch.load('./2024_08_14-05_29_56_PM/best_cp/modelk.pth',map_location='cuda:0'),strict=False)


#     model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

# model.to(device)
# model.fc = nn.Linear(model.fc.in_features, 10)
# model.to(device)


# criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
# criterion1 = nn.CrossEntropyLoss(reduction='none',label_smoothing=0.1)
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)




# # Model, Loss, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# resnet = models.resnet18(pretrained=False)
resnet = create_model()
# resnet.load_state_dict(torch.load('net2chest0.2withoutssl.pth'))
resnet.load_state_dict(torch.load('net2HAM0.2.pth'))

num_ftrs = resnet.linear.in_features
resnet.linear = nn.Identity()  # Replace the FC layer with an Identity layer
model = nn.Sequential(
    resnet,                    # ResNet-50 backbone without classification layer
    nn.Linear(num_ftrs, 512),   # First linear layer of the projection head
    nn.ReLU(),                  # ReLU activation function
    nn.Linear(512, 128)         # Second linear layer projecting to 128 dimensions
)
# model=resnet
# model.to(device)







# num_ftrs = resnet.fc.in_features
# resnet.fc = nn.Identity()  # Replace the FC layer with an Identity layer
# model = nn.Sequential(
#     resnet,                    # ResNet-50 backbone without classification layer
#     nn.Linear(num_ftrs, 512),   # First linear layer of the projection head
#     nn.ReLU(),                  # ReLU activation function
#     nn.Linear(512, 128)         # Second linear layer projecting to 128 dimensions
# )
# model.to(device)



# model = models.resnet50(pretrained=False)
# model.load_state_dict(torch.load('resnet50_weights.pth'))





'''

criterion = NTXentLoss(temperature=0.3)
# optimizer = optim.Adam(model.parameters(), lr=3e-4)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.95)

dataset = ExampleDataset1(data, new_labels,transform=SimCLRTransform)
dataloader = DataLoader(dataset, batch_size=400, shuffle=True,num_workers=4)
epochs = 15
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, _ in dataloader:
        img_1, img_2 = images  # Two augmented views of the same image

        # Move to device
        img_1, img_2 = img_1.to(device), img_2.to(device)

        # Forward pass
        z_i = model(img_1)
        z_j = model(img_2)

        # Compute loss
        loss = criterion(z_i, z_j)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(dataloader)}')

    
    
dataset = ExampleDataset1(data, new_labels,transform=SimCLRTransform1)
dataloader = DataLoader(dataset, batch_size=200, shuffle=True,num_workers=4)
epochs = 15
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, _ in dataloader:
        img_1, img_2 = images  # Two augmented views of the same image

        # Move to device
        img_1, img_2 = img_1.to(device), img_2.to(device)

        # Forward pass
        z_i = model(img_1)
        z_j = model(img_2)

        # Compute loss
        loss = criterion(z_i, z_j)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(dataloader)}')

    
    
    
torch.save(model.state_dict(), 'simclrsslvarybsdividemix1.pth')
'''

# model.load_state_dict(torch.load('simclrssl1.pth'))
# model.to(device)
# criterion = NTXentLoss(temperature=0.5)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.95)

# epochs = 20
# for epoch in range(epochs):
#     model.train()
#     running_loss = 0.0

#     for images, _ in dataloader:
#         img_1, img_2 = images  # Two augmented views of the same image

#         # Move to device
#         img_1, img_2 = img_1.to(device), img_2.to(device)

#         # Forward pass
#         z_i = model(img_1)
#         z_j = model(img_2)

#         # Compute loss
#         loss = criterion(z_i, z_j)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(dataloader)}')

# torch.save(model.state_dict(), 'simclrssladamsgd.pth')







# transform = BYOLTransform()
# dataset = CIFAR10BYOLDataset(data, new_labels,transform=transform)
# dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True)

# Define BYOL model and optimizer
# base_encoder = models.resnet50(pretrained=False)
# base_encoder.load_state_dict(torch.load('resnet50_weights.pth'))
# model = BYOL(base_encoder).to('cuda')
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Train for a number of epochs
# num_epochs = 10
# for epoch in range(num_epochs):
#     train_loss = train_byol(dataloader, model, optimizer)
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")




# torch.save(model.state_dict(), 'byolssl.pth')











# model.load_state_dict(torch.load('simclrsslvarybs.pth'))
# model.load_state_dict(torch.load('simclrsslvarybsdividemix1.pth'))
# model.load_state_dict(torch.load('byol.pth'))

# model.to(device)
# model.fc = nn.Sequential(
# #     nn.Dropout(0.3),  # Dropout with probability 0.3
#     nn.Linear(model.fc.in_features, 10)  # The output layer with 10 classes (for CIFAR-10)
# )
# model.to(device)


#simclr
model = nn.Sequential(
    model,                    # ResNet-50 backbone without classification layer
    nn.ReLU(),                  # ReLU activation function
    nn.Dropout(0.7),
    nn.Linear(128, 4)         # Second linear layer projecting to 128 dimensions
)




#sslafterdividemix
# num_ftrs = model.linear.in_features
# model.linear = nn.Identity()
# model = nn.Sequential(
#     model,                    # ResNet-50 backbone without classification layer
# #     nn.ReLU(),                  # ReLU activation function
# #     nn.Dropout(0.7),
#     nn.Linear(num_ftrs, 10)         # Second linear layer projecting to 128 dimensions
# )
# simclr end



# model.load_state_dict(torch.load('byolssl.pth'))


# pretrained_backbone = model.online_encoder

# Creating a fine-tuned model for CIFAR-10 classification
# model = FineTunedModel(pretrained_backbone).to('cuda')

# model = nn.Sequential(
#     model,                    # ResNet-50 backbone without classification layer
#     nn.ReLU(),                  # ReLU activation function
#     nn.Dropout(0.3),
#     nn.Linear(256, 10)         # Second linear layer projecting to 128 dimensions
# )



model.to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
criterion1 = nn.CrossEntropyLoss(reduction='none')

# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.95)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.95,weight_decay=5e-4)



criterion = FocalLoss(alpha=0.25, gamma=2.0)
criterion1 = FocalLoss(alpha=0.25, gamma=2.0,reduction='none')

dataset = NDataset(data, new_labels,transform=transform)
dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True,num_workers=4)

num_epochs = 5

for epoch in range(num_epochs):
#     dataset.update_confidence(model, device)
    model.train()
    running_loss = 0.0
    for inputs, targets, confidences in dataloader:
        inputs = inputs.to(device)
        targets=targets.to(device) 
        confidences=confidences.to(device)
        optimizer.zero_grad() 

        outputs = model(inputs)  
        loss = criterion(outputs, targets)
# #         confidences = torch.softmax(outputs, dim=1)
#         loss = training_criterion(outputs, targets, confidences)

        loss.backward() 
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()  
        # scheduler.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}')


val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
# model.train()
from sklearn.metrics import f1_score
f1 = f1_score(all_labels, all_predictions, average='macro')
print(f'Fold {fold + 1} F1 Score: {f1:.4f}')




# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.95)

# optimizer = optim.Adam(model.parameters(), lr=0.001)
stages = [2,3,4]
# stages=[2,4,7,9,11]
datasetpred = ExampleDataset(data, new_labels,transform=transform)
dataloaderpred = DataLoader(datasetpred, batch_size=bsize, shuffle=False,num_workers=4)
pseudo_labels=[]
confident_labels=[]
last_epoch=stages[-1]
ep=0;
for st in stages:
    model.train()
    num_epochs = st;
    for epoch in range(ep,num_epochs):
#         dataset.update_confidence(model, device)
        running_loss = 0.0
        for inputs, targets, confidences in dataloader:
            inputs = inputs.to(device)
            targets=targets.to(device) 
            confidences=confidences.to(device)
            optimizer.zero_grad()  # Zero the gradients

            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets)  # Compute loss
# #             confidences = torch.softmax(outputs, dim=1)
#             loss = training_criterion(outputs, targets, confidences)
        
            loss.backward()  # Backward pass
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()  # Optimize
            # scheduler.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{last_epoch}], Loss: {running_loss / len(dataloader):.4f}')
    all_predictions = []
    reliable_indice=[]
    model.eval()
#     pred, reliable_indices = pseudo_labeling(model, dataloaderpred, criterion1, device)
#     pseudo_labels.append(pred)
#     confident_labels.append(reliable_indices)

    with torch.no_grad():
        for inputs, labels in dataloaderpred: 
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass to get outputs
            outputs = model(inputs)

            # Get the predictions
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())

            # Calculate per-sample loss
            loss_per_sample = criterion1(outputs, labels)
            


            # print(loss_per_sample)
            reliable_indices = loss_per_sample < 0.25  # Example threshold for loss
            # print(reliable_indices)
            # Extend reliable indices list with boolean values indicating reliability
            reliable_indice.extend(reliable_indices.cpu().numpy())


    pseudo_labels.append(np.array(all_predictions))
    confident_labels.append(np.array(reliable_indice))
    ep=st;




val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
# model.train()
from sklearn.metrics import f1_score
f1 = f1_score(all_labels, all_predictions, average='macro')
print(f'Fold {fold + 1} F1 Score: {f1:.4f}')



result = common_elements_same_index(confident_labels)
print(len(result))
itr1_pseudo=new_labels
itr1_pseudo = replace_elements_at_indices(itr1_pseudo, result, pseudo_labels)

augmented_data = []
augmented_labels = []
for idx in result:
    input_data = data[idx]  # Get the data for the current index
    input_data = Image.fromarray(input_data)
    for _ in range(dmp):  # Perform 5 augmentations
        augmented_input = augmentation_transforms(input_data)  # Apply augmentations
        augmented_input = (augmented_input.permute(1, 2, 0).numpy()*255).astype(np.uint8)
        augmented_data.append(augmented_input)
        augmented_labels.append(itr1_pseudo[idx])  # Use the pseudo-label for augmented data

augmented_labels = np.array(augmented_labels)

# Concatenate the original data and augmented data
data = np.concatenate((data, np.array(augmented_data)), axis=0)
itr1_pseudo = np.concatenate((itr1_pseudo, augmented_labels), axis=0)


dataset = NDataset(data, itr1_pseudo,transform=transform)
dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True,num_workers=4)
datasetpred = ExampleDataset(data, itr1_pseudo,transform=transform)
dataloaderpred = DataLoader(datasetpred, batch_size=bsize, shuffle=False,num_workers=4)
p=[]
a=[]
optimizer = optim.Adam(model.parameters(), lr=0.00001)
augmented_epochs=[1,3,5]
for i in range(3):
    print(f'Iteration: {i}')
    stages = [2,5,7]
#     stages = [3,5,6,8,10]
    pseudo_labels1=[]
    confident_labels=[]
    last_epoch=stages[-1]
    ep=0;
    for st in stages:
        model.train()
        num_epochs = st;
        for epoch in range(ep,num_epochs):
#             dataset.update_confidence(model, device)
            running_loss = 0.0
            for inputs, targets, confidences in dataloader:
                inputs = inputs.to(device)
                targets=targets.to(device) 
                confidences=confidences.to(device)
                optimizer.zero_grad()  # Zero the gradients

                outputs = model(inputs)  # Forward pass
                loss = criterion(outputs, targets)  # Compute loss
# #                 confidences = torch.softmax(outputs, dim=1)
#                 loss = training_criterion(outputs, targets, confidences)

                loss.backward()  # Backward pass
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()  # Optimize
                # scheduler.step()
                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{last_epoch}], Loss: {running_loss / len(dataloader):.4f}')
        all_predictions = []
        reliable_indice=[]
        model.eval()
#         pred, reliable_indices = pseudo_labeling(model, dataloaderpred, criterion1, device)
#         pseudo_labels1.append(pred)
#         confident_labels.append(reliable_indices)
        with torch.no_grad():
            for inputs, labels in dataloaderpred: 
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass to get outputs
                outputs = model(inputs)

                # Get the predictions
                _, predicted = torch.max(outputs, 1)
                all_predictions.extend(predicted.cpu().numpy())

                # Calculate per-sample loss
                loss_per_sample = criterion1(outputs, labels)
            

                # Set a loss threshold (lower loss means more reliable)
                reliable_indices = loss_per_sample < 0.25  # Example threshold for loss

                # Extend reliable indices list with boolean values indicating reliability
                reliable_indice.extend(reliable_indices.cpu().numpy())

        pseudo_labels1.append(np.array(all_predictions))
        confident_labels.append(np.array(reliable_indice))
        ep=st;
    result1 = common_elements_same_index(confident_labels)
    p.append(result1)
    a.append(pseudo_labels1)
    print(len(result1))
    itr1_pseudo = replace_elements_at_indices(itr1_pseudo, result1, pseudo_labels1)
    
    if i <2:
        augmented_data = []
        augmented_labels = []
        for idx in result1:
            input_data = data[idx]  # Get the data for the current index
            input_data = Image.fromarray(input_data)
            for _ in range(dmp):  # Perform 5 augmentations
                augmented_input = augmentation_transforms(input_data)  # Apply augmentations
                augmented_input = (augmented_input.permute(1, 2, 0).numpy()*255).astype(np.uint8)
                augmented_data.append(augmented_input)
                augmented_labels.append(itr1_pseudo[idx])  # Use the pseudo-label for augmented data

        augmented_labels = np.array(augmented_labels)

        # Concatenate the original data and augmented data
        data = np.concatenate((data, np.array(augmented_data)), axis=0)
        itr1_pseudo = np.concatenate((itr1_pseudo, augmented_labels), axis=0)
    
    
    
    dataset = NDataset(data, itr1_pseudo,transform=transform)
    dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True,num_workers=4)
    datasetpred = ExampleDataset(data, itr1_pseudo,transform=transform)
    dataloaderpred = DataLoader(datasetpred, batch_size=bsize, shuffle=False,num_workers=4)



    val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    # model.train()
    from sklearn.metrics import f1_score
    f1 = f1_score(all_labels, all_predictions, average='macro')
    print(f'Fold {fold + 1} F1 Score: {f1:.4f}')
    if(i==1):
        optimizer = optim.Adam(model.parameters(), 0.000001)

'''    
augmented_epochs=[1,2,4]
for i in range(0):
    print(f'Iteration: {2+i}')
    stages = [3,6,10]
    pseudo_labels1=[]
    confident_labels=[]
    last_epoch=stages[-1]
    ep=0;
    for st in stages:
        model.train()
        num_epochs = st;
        for epoch in range(ep,num_epochs):
            running_loss = 0.0
            for inputs, targets in dataloader:
                inputs = inputs.to(device)
                targets=targets.to(device) 
                confidences=confidences.to(device)
                optimizer.zero_grad()  # Zero the gradients

                outputs = model(inputs)  # Forward pass
                loss = criterion(outputs, targets)  # Compute loss

                loss.backward()  # Backward pass
                optimizer.step()  # Optimize
                # scheduler.step()
                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{last_epoch}], Loss: {running_loss / len(dataloader):.4f}')
        all_predictions = []
        reliable_indice=[]
        model.eval()
        with torch.no_grad():
            for inputs, labels in dataloaderpred: 
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass to get outputs
                outputs = model(inputs)

                # Get the predictions
                _, predicted = torch.max(outputs, 1)
                all_predictions.extend(predicted.cpu().numpy())

                # Calculate per-sample loss
                loss_per_sample = criterion1(outputs, labels) 

                # Set a loss threshold (lower loss means more reliable)
                reliable_indices = loss_per_sample < 0.25  # Example threshold for loss

                # Extend reliable indices list with boolean values indicating reliability
                reliable_indice.extend(reliable_indices.cpu().numpy())

        pseudo_labels1.append(np.array(all_predictions))
        confident_labels.append(np.array(reliable_indice))
        ep=st;
    result1 = common_elements_same_index(confident_labels)
    p.append(result1)
    a.append(pseudo_labels1)
    print(len(result1))
    itr1_pseudo = replace_elements_at_indices(itr1_pseudo, result1, pseudo_labels1)
    
#     if i in augmented_epochs:
    augmented_data = []
    augmented_labels = []
    for idx in result1:
        input_data = data[idx]  # Get the data for the current index
        input_data = Image.fromarray(input_data)
        for _ in range(dmp):  # Perform 5 augmentations
            augmented_input = augmentation_transforms(input_data)  # Apply augmentations
            augmented_input = (augmented_input.permute(1, 2, 0).numpy()*255).astype(np.uint8)
            augmented_data.append(augmented_input)
            augmented_labels.append(itr1_pseudo[idx])  # Use the pseudo-label for augmented data

    augmented_labels = np.array(augmented_labels)

    # Concatenate the original data and augmented data
    data = np.concatenate((data, np.array(augmented_data)), axis=0)
    itr1_pseudo = np.concatenate((itr1_pseudo, augmented_labels), axis=0)
    
    
    dataset = NDataset(data, itr1_pseudo,transform=transform)
    dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True,num_workers=4)
    datasetpred = ExampleDataset(data, itr1_pseudo,transform=transform)
    dataloaderpred = DataLoader(datasetpred, batch_size=bsize, shuffle=False,num_workers=4)


    val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    # model.train()
    from sklearn.metrics import f1_score
    f1 = f1_score(all_labels, all_predictions, average='macro')
    print(f'Fold {fold + 1} F1 Score: {f1:.4f}')
'''
net2=model
    
# for i in range(1):
#     print(f'Iteration: {2+i}')
#     stages = [2,5]
#     pseudo_labels1=[]
#     confident_labels=[]
#     last_epoch=stages[-1]
#     ep=0;
#     for st in stages:
#         model.train()
#         num_epochs = st;
#         for epoch in range(ep,num_epochs):
#             running_loss = 0.0
#             for inputs, targets in dataloader:
#                 inputs = inputs.to(device)
#                 targets=targets.to(device) 
#                 optimizer.zero_grad()  # Zero the gradients

#                 outputs = model(inputs)  # Forward pass
#                 loss = criterion(outputs, targets)  # Compute loss

#                 loss.backward()  # Backward pass
#                 optimizer.step()  # Optimize
#                 # scheduler.step()
#                 running_loss += loss.item()

#             print(f'Epoch [{epoch + 1}/{last_epoch}], Loss: {running_loss / len(dataloader):.4f}')
#         all_predictions = []
#         reliable_indice=[]
#         model.eval()
#         with torch.no_grad():
#             for inputs, labels in dataloaderpred: 
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 # Forward pass to get outputs
#                 outputs = model(inputs)

#                 # Get the predictions
#                 _, predicted = torch.max(outputs, 1)
#                 all_predictions.extend(predicted.cpu().numpy())

#                 # Calculate per-sample loss
#                 loss_per_sample = criterion1(outputs, labels) 

#                 # Set a loss threshold (lower loss means more reliable)
#                 reliable_indices = loss_per_sample < 0.25  # Example threshold for loss

#                 # Extend reliable indices list with boolean values indicating reliability
#                 reliable_indice.extend(reliable_indices.cpu().numpy())

#         pseudo_labels1.append(np.array(all_predictions))
#         confident_labels.append(np.array(reliable_indice))
#         ep=st;
#     result1 = common_elements_same_index(confident_labels)
#     p.append(result1)
#     a.append(pseudo_labels1)
#     print(len(result1))
#     itr1_pseudo = replace_elements_at_indices(itr1_pseudo, result1, pseudo_labels1)
#     dataset = ExampleDataset(data, itr1_pseudo,transform=transform)
#     dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True)
#     datasetpred = ExampleDataset(data, itr1_pseudo,transform=transform)
#     dataloaderpred = DataLoader(dataset, batch_size=bsize, shuffle=False)


#     val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
#     val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
#     model.eval()
#     all_predictions = []
#     all_labels = []

#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)
#             outputs = model(inputs)
#             _, predicted = torch.max(outputs, 1)
#             all_predictions.extend(predicted.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#     # model.train()
#     from sklearn.metrics import f1_score
#     f1 = f1_score(all_labels, all_predictions, average='macro')
#     print(f'Fold {fold + 1} F1 Score: {f1:.4f}')

# val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
# val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
# model.eval()
# all_predictions = []
# all_labels = []

# with torch.no_grad():
#     for inputs, labels in val_loader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         all_predictions.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())
# # model.train()
# from sklearn.metrics import f1_score
# f1 = f1_score(all_labels, all_predictions, average='macro')
# print(f'Fold {fold + 1} F1 Score: {f1:.4f}')


Epoch [1/5], Loss: 0.1555
Epoch [2/5], Loss: 0.1396
Epoch [3/5], Loss: 0.1369
Epoch [4/5], Loss: 0.1371
Epoch [5/5], Loss: 0.1327
Fold 1 F1 Score: 0.2348
Epoch [1/4], Loss: 0.1341
Epoch [2/4], Loss: 0.1320
Epoch [3/4], Loss: 0.1331
Epoch [4/4], Loss: 0.1308
Fold 1 F1 Score: 0.2523
2420
Iteration: 0
Epoch [1/7], Loss: 0.0822
Epoch [2/7], Loss: 0.0736
Epoch [3/7], Loss: 0.0724
Epoch [4/7], Loss: 0.0702
Epoch [5/7], Loss: 0.0697
Epoch [6/7], Loss: 0.0696
Epoch [7/7], Loss: 0.0683
7219
Fold 1 F1 Score: 0.2525
Iteration: 1
Epoch [1/7], Loss: 0.0442
Epoch [2/7], Loss: 0.0428
Epoch [3/7], Loss: 0.0424
Epoch [4/7], Loss: 0.0410
Epoch [5/7], Loss: 0.0413
Epoch [6/7], Loss: 0.0404
Epoch [7/7], Loss: 0.0404
21637
Fold 1 F1 Score: 0.2490
Iteration: 2
Epoch [1/7], Loss: 0.0288
Epoch [2/7], Loss: 0.0279
Epoch [3/7], Loss: 0.0278
Epoch [4/7], Loss: 0.0275
Epoch [5/7], Loss: 0.0272
Epoch [6/7], Loss: 0.0271
Epoch [7/7], Loss: 0.0270
64826
Fold 1 F1 Score: 0.2471


In [43]:
data = data1
new_labels = new_labels1
val_data = test_images
val_labels = test_labels
val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs1 = net1(inputs)
        outputs2 = net2(inputs)
        outputs = (outputs1 + outputs2) / 2
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
# model.train()
from sklearn.metrics import f1_score
f1 = f1_score(all_labels, all_predictions, average='macro')
print(f'Fold {fold + 1} F1 Score: {f1:.4f}')

Fold 1 F1 Score: 0.2242


In [44]:
# data = data1
# new_labels = new_labels1
# val_data = test_images
# val_labels = test_labels
# val_dataset = ExampleDataset(val_data, val_labels,transform=transform)
# val_loader = DataLoader(val_dataset, batch_size=bsize, shuffle=False)
# model.eval()
# all_predictions = []
# all_labels = []

# with torch.no_grad():
#     for inputs, labels in val_loader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         all_predictions.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())
# # model.train()
# from sklearn.metrics import f1_score
# f1 = f1_score(all_labels, all_predictions, average='macro')
# print(f'Fold {fold + 1} F1 Score: {f1:.4f}')

In [45]:
# def common_elements_same_index(arrays):
#     if not arrays:
#         return []

#     # Number of arrays
#     num_arrays = len(arrays)
    
#     # Length of the first array
#     num_elements = len(arrays[0])

#     # Store indices where all arrays have the same element
#     common_indices = []
    
#     # Iterate over each index
#     for i in range(num_elements):
#         # Get the element at index i in the first array
#         element = arrays[0][i]
        
#         # Check if all arrays have the same element at index i
#         if all(array[i] == element for array in arrays):
#             common_indices.append(i)
    
#     return common_indices


# result1 = common_elements_same_index(pseudo_labels)



In [46]:
# len(result1)

In [47]:
# # num_epochs = 5
# running_loss=50;
# epoch=0;
# while True:
#     running_loss = 0.0
#     for inputs, targets in dataloader:
#         inputs = inputs.to(device)
#         targets=targets.to(device) 
#         optimizer.zero_grad()  

#         outputs = model(inputs) 
#         loss = criterion(outputs, targets)  

#         loss.backward() 
#         optimizer.step() 

#         running_loss += loss.item()

# #   print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}')
#     print(f'Epoch [{epoch + 1}], Loss: {running_loss / len(dataloader):.4f}')
#     if running_loss / len(dataloader)<0.6:
#         break;
#     epoch=epoch+1;


In [48]:
# # Save the model
# torch.save(model.state_dict(), 'resnet50_weights_cifar10_wbyol_loss.pth')

In [49]:
# import torch
# from torch.utils.data import DataLoader
# import torchvision.transforms as transforms
# from torchvision.datasets import MNIST
# from torchvision import models
# import torch.nn as nn


# resnet = models.resnet50(pretrained=False)
# # resnet.load_state_dict(torch.load('resnet50_weights.pth'))

# # Remove the classification layer (fully connected layer)
# num_ftrs = resnet.fc.in_features
# resnet.fc = nn.Identity()  # Replace the FC layer with an Identity layer

# # Define the projection head (2-layer MLP) and include it with the ResNet model
# model = nn.Sequential(
#     resnet,                    # ResNet-50 backbone without classification layer
#     nn.Linear(num_ftrs, 512),   # First linear layer of the projection head
#     nn.ReLU(),                  # ReLU activation function
#     nn.Linear(512, 128)         # Second linear layer projecting to 128 dimensions
# )

# model = nn.Sequential(
#     model,                    # ResNet-50 backbone without classification layer
#     nn.ReLU(),                  # ReLU activation function
#     nn.Dropout(0.3),
#     nn.Linear(128, 10)         # Second linear layer projecting to 128 dimensions
# )

# # model = models.resnet50(pretrained=True)
# # model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
# # model.fc = nn.Linear(model.fc.in_features, 10)
# model.load_state_dict(torch.load('resnet50_weights_cifar10_wbyol_loss.pth'))
# model.eval()

# # transform = transforms.Compose([
# # #     transforms.ToPILImage(),  
# #     transforms.ToTensor(),
# #     transforms.Normalize((0.467, 0.459, 0.519), (0.207, 0.233, 0.261))
# # ])

# test_images = np.load('data/cifar10/test_images.npy')
# test_labels = np.load('data/cifar10/test_labels.npy')

# # test_dataset = ExampleDataset(test_images, test_labels,transform)
# test_images=test_images.reshape(-1, 3, 32, 32)
# data=test_images
# data=data.reshape(-1, 3, 32, 32)
# data=data.transpose((0, 2, 3, 1))
# testl = torch.from_numpy(test_labels)
# test_dataset=ExampleDataset(data,testl,transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=bsize, shuffle=False)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# all_predictions = []
# all_labels = []

# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs = inputs.to(device)
#         labels=labels.to(device) 
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         all_predictions.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())

# print(np.array(all_predictions[:1000])-np.array(testl[:1000]))



In [50]:

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(np.array(all_labels), np.array(all_predictions))

print(cm)


[[87  0 55  0]
 [29  0 97  0]
 [57  0 81  0]
 [44  0 22  0]]


In [51]:
# from torchmetrics.classification import MulticlassF1Score
# f1_score = MulticlassF1Score(num_classes=10, average='macro')

# all_predictions = torch.tensor(all_predictions)
# all_labels = torch.tensor(all_labels)

# f1_score.update(all_predictions, all_labels)
# f1 = f1_score.compute()

# print(f'F1 Score: {f1:.4f}')

from sklearn.metrics import f1_score
f1 = f1_score(all_labels, all_predictions, average='macro')
print(f1)
# print(f'Fold {fold + 1} F1 Score: {f1:.4f}')

0.22422335154904421


In [52]:
from torchmetrics.classification import MulticlassAccuracy
import torch

# Assuming all_predictions and all_labels are defined as lists or arrays
all_predictions = torch.tensor(all_predictions)
all_labels = torch.tensor(all_labels)

# Initialize accuracy metric for multiclass classification
accuracy_metric = MulticlassAccuracy(num_classes=10)

# Update metric with predictions and labels
accuracy_metric.update(all_predictions, all_labels)

# Compute the accuracy
accuracy = accuracy_metric.compute()

# Print the accuracy
print(f'Accuracy: {accuracy:.4f}')


NOTE! Installing ujson may make loading annotations faster.
Accuracy: 0.2999


In [53]:
# import torch
# import numpy as np
# from torchvision import transforms, datasets
# from torch.utils.data import DataLoader, Dataset
# import torchvision.models as models
# import torch.nn as nn

# model = models.resnet50(pretrained=False)
# model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.fc = nn.Linear(model.fc.in_features, 10)
# model.load_state_dict(torch.load('resnet50_weights_cifar10.pth'))
# model.eval()

# # transform = transforms.Compose([
# # #     transforms.ToPILImage(),  
# #     transforms.ToTensor(),
# #     transforms.Normalize((0.467, 0.459, 0.519), (0.207, 0.233, 0.261))
# # ])

# class CustomDataset(Dataset):
#     def __init__(self, data, transform=None):
#         self.data = data
#         self.transform = transform

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, index):
#         x = self.data[index]
#         if self.transform:
#             x = self.transform(x)
#         return x


# original_images = np.load('data/cifar10/train_images.npy')
# data=original_images
# data=data.reshape(-1, 3, 32, 32)
# data=data.transpose((0, 2, 3, 1))

# dataset = CustomDataset(images, transform=transform)
# dataloader = DataLoader(dataset, batch_size=bsize, shuffle=False)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# all_predictions = []

# with torch.no_grad():
#     for inputs in dataloader:
#         inputs = inputs.to(device)
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         all_predictions.extend(predicted.cpu().numpy())

# pseudo_labels = np.array(all_predictions)

# np.save('pseudo_labels.npy', pseudo_labels)

# print(f'Pseudo labels have been saved to pseudo_labels.npy')


In [54]:
# original_images = np.load('data/cifar10/train_images.npy')
# pseudo_labels = np.load('pseudo_labels.npy')
# data=original_images
# data=data.reshape(-1, 3, 32, 32)
# data=data.transpose((0, 2, 3, 1))
# targets = torch.from_numpy(pseudo_labels).to(device)

In [55]:
# dataset = ExampleDataset(data, targets,transform=transform)
# dataloader = DataLoader(dataset, batch_size=bsize, shuffle=True)


In [56]:
# model = models.resnet50(pretrained=False)
# model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.fc = nn.Linear(model.fc.in_features, 10)

In [57]:
# criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
# optimizer = optim.Adam(model.parameters(), lr=0.001)


In [58]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

In [59]:
# num_epochs = 5

# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for inputs, targets in dataloader:
#         inputs = inputs.to(device)
#         targets=targets.to(device) 
#         optimizer.zero_grad() 

#         outputs = model(inputs)  
#         loss = criterion(outputs, targets)  

#         loss.backward() 
#         optimizer.step()  
#         # scheduler.step()
#         running_loss += loss.item()

#     print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}')


In [60]:
# torch.save(model.state_dict(), 'resnet50_weights_cifar10.pth')

In [61]:
# import torch
# from torch.utils.data import DataLoader
# import torchvision.transforms as transforms
# from torchvision.datasets import MNIST
# from torchvision import models
# import torch.nn as nn

# model = models.resnet50(pretrained=False)
# model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.fc = nn.Linear(model.fc.in_features, 10)
# model.load_state_dict(torch.load('resnet50_weights_cifar10.pth'))
# model.eval()

# # transform = transforms.Compose([
# # #     transforms.ToPILImage(),  
# #     transforms.ToTensor(),
# #     transforms.Normalize((0.467, 0.459, 0.519), (0.207, 0.233, 0.261))
# # ])

# test_images = np.load('data/cifar10/test_images.npy')
# test_labels = np.load('data/cifar10/test_labels.npy')

# test_images=test_images.reshape(-1, 3, 32, 32)
# data=test_images
# data=data.reshape(-1, 3, 32, 32)
# data=data.transpose((0, 2, 3, 1))
# testl = torch.from_numpy(test_labels)
# test_dataset=ExampleDataset(data,testl,transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=bsize, shuffle=False)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# all_predictions = []
# all_labels = []

# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs = inputs.to(device)
#         labels=labels.to(device) 
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         all_predictions.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())

# print(np.array(all_predictions[:1000])-np.array(testl[:1000]))



In [62]:

# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(np.array(all_labels), np.array(all_predictions))

# print(cm)


In [63]:
# from torchmetrics.classification import MulticlassF1Score
# f1_score = MulticlassF1Score(num_classes=10, average='macro')

# all_predictions = torch.tensor(all_predictions)
# all_labels = torch.tensor(all_labels)

# f1_score.update(all_predictions, all_labels)
# f1 = f1_score.compute()

# print(f'F1 Score: {f1:.4f}')